In [1]:
! pip install -q kaggle

In [14]:
!pip install transformers


     |████████████████████████████████| 2.6 MB 8.4 MB/s 
     |████████████████████████████████| 636 kB 51.7 MB/s 
     |████████████████████████████████| 3.3 MB 58.4 MB/s 
     |████████████████████████████████| 895 kB 57.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [25]:
!pip install unidecode
!pip install torch
!pip install tqdm

     |████████████████████████████████| 241 kB 7.7 MB/s 


In [2]:
from google.colab import files

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gabib3b","key":"817d7e169db4cbef867b22907320144c"}'}

In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle competitions download -c home-depot-product-search-relevance

 66% 18.0M/27.2M [00:00<00:00, 13.8MB/s]
100% 27.2M/27.2M [00:00<00:00, 39.8MB/s]
  0% 0.00/105k [00:00<?, ?B/s]
100% 105k/105k [00:00<00:00, 101MB/s]
  0% 0.00/227k [00:00<?, ?B/s]
100% 227k/227k [00:00<00:00, 73.6MB/s]
100% 4.74M/4.74M [00:00<00:00, 11.1MB/s]

  0% 0.00/2.51M [00:00<?, ?B/s]
100% 2.51M/2.51M [00:00<00:00, 82.7MB/s]
 49% 17.0M/34.8M [00:00<00:00, 33.2MB/s]
100% 34.8M/34.8M [00:00<00:00, 59.1MB/s]


In [19]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import nltk 


In [9]:
!ls

attributes.csv.zip	      relevance_instructions.docx  test.csv.zip
kaggle.json		      sample_data		   train.csv.zip
product_descriptions.csv.zip  sample_submission.csv.zip


In [10]:
att_df= pd.read_csv('attributes.csv.zip')
desc_df= pd.read_csv('product_descriptions.csv.zip')
train_df= pd.read_csv('train.csv.zip', encoding='latin-1')
test_df = pd.read_csv('test.csv.zip',encoding='latin-1')



In [11]:

desc_df = desc_df.fillna(0)
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
att_df = att_df.fillna(0)

desc_df['product_uid'] = desc_df['product_uid'].astype(np.int64)
train_df['product_uid'] = train_df['product_uid'].astype(np.int64)
test_df['product_uid'] = test_df['product_uid'].astype(np.int64)
att_df['product_uid'] = att_df['product_uid'].astype(np.int64)

In [13]:
train_df.sample(2)

,id,product_uid,product_title,search_term,relevance
38221,116794,138961,Charlotte Pipe 2 in. PVC Sch. 40 Plug,2 in pvc pipe incresers,2.0
19087,58961,115711,Glacier Bay Newport 37 in. AB Engineered Compo...,vanity top 37,3.0


In [27]:
np.min(train_df['relevance'].tolist()), np.max(train_df['relevance'].tolist())

(1.0, 3.0)

In [28]:
train_df['my_relevance'] = train_df['relevance'].apply(lambda x: (x - 1.0)/2.0)

In [30]:
np.min(train_df['my_relevance'].tolist()), np.max(train_df['my_relevance'].tolist())

(0.0, 1.0)

In [33]:
train_df['my_relevance2'] = train_df['my_relevance'].apply(lambda x: ((x - 0.0)/1.0) * 2.0 + 1)

In [34]:
train_df[['relevance', 'my_relevance', 'my_relevance2']].sample(4)

,relevance,my_relevance,my_relevance2
58211,1.33,0.165,1.33
10005,1.67,0.335,1.67
46037,2.33,0.665,2.33
24224,1.33,0.165,1.33


In [ ]:
train_df.describe()

,id,product_uid,relevance
count,74067.000000,74067.000000,74067.000000
mean,112385.709223,142331.911553,2.381634
std,64016.573650,30770.774864,0.533984
min,2.000000,100001.000000,1.000000
25%,57163.500000,115128.500000,2.000000
50%,113228.000000,137334.000000,2.330000
75%,168275.500000,166883.500000,3.000000
max,221473.000000,206650.000000,3.000000


In [ ]:
desc_df.drop_duplicates(["product_uid"]).shape[0]

124428

In [ ]:
pd.merge(train_df, desc_df, how='left', on='product_uid').shape[0]

74067

In [16]:
train_df.shape[0]

74067

In [20]:
def clean_text(string):
  if string is None:
    return None

  string = string.strip().lower()
  string = unidecode.unidecode(string)
  string = string.replace("'", stringutils.EMPTY_STRING)
  string = re.sub(r"\'n", stringutils.SPACE, string)
  string = re.sub(r"\'s", " 's", string)
  string = re.sub(r"\'ve", " 've", string)
  string = re.sub(r"n\'t", " n't", string)
  string = re.sub(r"\'re", " 're", string)
  string = re.sub(r"\'d", " 'd", string)
  string = re.sub(r"\'ll", " 'll", string)
  string = re.sub(r",", " , ", string)
  string = re.sub(r"!", " ! ", string)
  string = re.sub(r"\(", " ( ", string)
  string = re.sub(r"\)", " ) ", string)
  string = re.sub(r"\?", r" \? ", string)
  string = re.sub(r"\s{2,}", stringutils.SPACE, string)
  string = re.sub(r" +", stringutils.SPACE, string)
  string = re.sub(r"/", " / ", string)
  string = re.sub(r"\|", " | ", string)
  string = re.sub(r"-", " - ", string)
  string = re.sub(r"\+", " + ", string)
  string = re.sub(r"\\\\", stringutils.SPACE, string)

  return string.strip()


In [23]:
def tokenize(text):
  
  processed_text = clean_text(text)
  tokens = nltk.word_tokenize(processed_text)

  return tokens

In [24]:
tokenize("gabi went home")

NameError: ignored